<img src="./lecture_image/00_title.png">

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:40px; text-align: center;"><font size=4 color=blue><b>[4차시] 학습목표</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>
        
○ 기본적인 FastAPI 사용법 학습하기<br>
○ 웹 화면에 카메라 영상 출력하기<br>
○ 웹 출력 영상을 Gray 이미지로 출력하기        
</font></td></tr>   
</table>

# FastAPI 설치 및 서버 실행하기

- <font color=red>FastAPI</font> : Sebastián Ramírez란 사람이 만든 파이썬 기반 오픈소스 웹 프레임워크
  - 파이썬 3.6 이상 
  - 빠른(고성능) API 서버 웹 프레임워크
  - Flask나 Django보다 단순하고 배우기 쉬움 
  - GitHub : https://github.com/tiangolo/fastapi
  - 튜토리얼 : https://fastapi.tiangolo.com/ko/
  - 다양한 형태의 Response : https://fastapi.tiangolo.com/ko/advanced/custom-response/


- ASGI(Asynchronous Server Gateway Interface) : 비동기 Python 웹 서버  
  - 동기(synchronous) 처리 : 특정 작업이 끝나면 다음 작업을 처리하는 순차처리 방식
    - 파이썬은 기본적으로 동기 방식으로 동작하는 - 
  - 비동기(asynchronous) 처리 : 여러 작업을 처리하도록 예약한 뒤 작업이 끝나면 결과를 받는 방식
    - 스레드 하나로 동시 처리를 하기 위한 것

  <img src="./lecture_image/04_fastapi06.png" width=50%>  
  

- <font color=red>pyngrok</font> : 방화벽을 넘어서 외부에서 로컬에 접속 가능하게 하는 터널 프로그램


- <font color=red>asyncio</font> : Python 표준 라이브러리와 함께 제공되는 비동기 I/O 프레임워크로 파이썬 3.4에서 추가


- <font color=red>async / await</font> : 비동기(Asynchronous) 처리를 위해 파이썬 3.5에서 추가한 구문
  - async : 비동기 함수를 생성하는 명령
  - await : 비동기 함수를 호출하는 명령
  
  
- <font color=red>nest_asyncio</font> : 비동기 방식에서 이벤트가 중첩되는 것을 허용하는 기능을 제공하는 라이브러리
  
  
- <font color=red>uvloop</font> : asyncio를 대체하기 위해 제작
  - 동기 함수 내에서 비동기 함수를 호출하려면 asyncio 라이브러리의 이벤트 루프를 이용하는 데 이를 대신하는 기능
  
  - Cython으로 작성
  - libuv 위에 구축 (libuv : nodejs에서 사용하는 고성능 다중 플랫폼 비동기 I / O 라이브러리)
  - 성능상에서 nodejs, gevent 및 기타 Python 비동기 프레임 워크보다 2배 이상 빠름
    
  
- <font color=red>uvicorn</font> : uvloop 및 httptools를 사용하여 만든 빠른 ASGI (Asynchronous Server Gateway Inteface) 서버
  - 다양한 관리 기능으로 fastapi와 함께 사용
  - HTTP / 1.1 및 WebSocket을 지원

## 동기 프로그래밍

In [1]:
#실습
import time

def find_users_sync(n):
    for i in range(1,n+1):
        print(f"{n}명 중 {i}번째 사용자 조회중 ...")
        time.sleep(1)
    print(f"> 총 {n}명 사용자 동기 조회 완료!!")

find_users_sync(5)

5명 중 1번째 사용자 조회중 ...
5명 중 2번째 사용자 조회중 ...
5명 중 3번째 사용자 조회중 ...
5명 중 4번째 사용자 조회중 ...
5명 중 5번째 사용자 조회중 ...
> 총 5명 사용자 동기 조회 완료!!


- 3개의 요청을 동기 처리

In [2]:
#실습
def process_sync():
    start = time.time()
    find_users_sync(3)
    find_users_sync(2)
    find_users_sync(1)
    end = time.time()
    print(f">> 동기방식으로 처리한 시간 : {end-start}")
    
process_sync()

3명 중 1번째 사용자 조회중 ...
3명 중 2번째 사용자 조회중 ...
3명 중 3번째 사용자 조회중 ...
> 총 3명 사용자 동기 조회 완료!!
2명 중 1번째 사용자 조회중 ...
2명 중 2번째 사용자 조회중 ...
> 총 2명 사용자 동기 조회 완료!!
1명 중 1번째 사용자 조회중 ...
> 총 1명 사용자 동기 조회 완료!!
>> 동기방식으로 처리한 시간 : 6.0332653522491455


## 비동기 프로그래밍

In [4]:
!pip install nest_asyncio fastapi uvicorn

  Using cached fastapi-0.92.0-py3-none-any.whl (56 kB)
     ---------------------------------------- 56.9/56.9 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 66.4/66.4 kB ? eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 9.8 MB/s eta 0:00:00


In [5]:
#실습
import time
import asyncio

# async : 비동기 함수 선언 (코루틴 함수)
async def find_users_sync(n):
    for i in range(1,n+1):
        print(f"{n}명 중 {i}번째 사용자 조회중 ...")
        await asyncio.sleep(1) # 비동기용 타이머
    print(f"> 총 {n}명 사용자 동기 조회 완료!!")

find_users_sync(5)

<coroutine object find_users_sync at 0x00000211C5E5F440>

- await asyncio.wait() : 여러 개의 코루틴 함수를 호출
- asyncio.run() :  코루틴 함수를 실행

In [10]:
#실습
import nest_asyncio

# 다중 작업 간의 메시지 교환
nest_asyncio.apply()

async def process_async():
    start = time.time()
    
    await asyncio.wait([find_users_sync(3),
                       find_users_sync(2),
                       find_users_sync(1)])
    end = time.time()
    print(f">>비동기 방식으로 처리한 시간 : {end-start}")

asyncio.run(process_async())

C:\Users\smhrd\AppData\Local\Temp\ipykernel_443456\453634581.py:10: DeprecationWarning: The explicit passing of coroutine objects to asyncio.wait() is deprecated since Python 3.8, and scheduled for removal in Python 3.11.
  await asyncio.wait([find_users_sync(3),


2명 중 1번째 사용자 조회중 ...
1명 중 1번째 사용자 조회중 ...
3명 중 1번째 사용자 조회중 ...
2명 중 2번째 사용자 조회중 ...
> 총 1명 사용자 동기 조회 완료!!
3명 중 2번째 사용자 조회중 ...
> 총 2명 사용자 동기 조회 완료!!
3명 중 3번째 사용자 조회중 ...
> 총 3명 사용자 동기 조회 완료!!
>>비동기 방식으로 처리한 시간 : 3.0188686847686768


##  FastAPI 설치하기

In [11]:
!pip install fastapi uvicorn pyngrok nest_asyncio

     -------------------------------------- 761.3/761.3 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19772 sha256=e29c3da93bb8d11cb244bfed6e02676a8122d8df375e41d1c5306ac70a9823ea
  Stored in directory: c:\users\smhrd\appdata\local\pip\cache\wheels\f6\89\59\49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


## FastAPI 서버 실행하기

- 참고 : https://blog.naver.com/PostView.naver?blogId=duqrlwjddns1&logNo=222268635237&parentCategoryNo=&categoryNo=&viewDate=&isShowPopularPosts=false&from=postView


- 비동기 처리(asyncio)는 Jupyter Notebook 환경에서 지원하지 않음


- <font color=red>app = FastAPI()</font> : FastAPI 객체를 app 변수에 할당
- <font color=red>nest_asyncio.apply()</font> : 이벤트 루프
- <font color=red>@app.get("/")</font> : GET 메소드로 root url을 접속하는 경우
- <font color=red>async def root()</font> : root() 비동기 함수 (native coroutine)을 만듬
- <font color=red>return {"message": "Hello World"}</font> : message 변수로 해당 값을 JSON 형태로 반환
- <font color=red>uvicorn.run(app, host="127.0.0.1", port=8000)</font> : app 객체에 IP와 Port 설정

<img src="./lecture_image/04_fastapi01.png" width=50%> 

In [16]:
#실습
from fastapi import FastAPI
import nest_asyncio
import uvicorn

# 웹 서버 초기화
app = FastAPI()
nest_asyncio.apply()

# 라우터 설정(get방식 , root)
@app.get("/")
async def root():
    return{"message":"야이야"}

# 메인함수라면 (파이썬을 단독으로 실행했다면)
if __name__ == "__main__":
    # 서버시작
    uvicorn.run(app, host="218.149.140.96",port=9000)

INFO:     Started server process [443456]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://218.149.140.96:9000 (Press CTRL+C to quit)


INFO:     218.149.140.96:55672 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [443456]


### URL를 함수로 연결하기

- <font color=red>@app.get("/hello")</font>
  - URL에  함수명을 설정
  - 브라우저에서 IP와 포트로 서버 접속하고 URL로 함수명을 넘김
  
  <img src="./lecture_image/04_fastapi02.png" width=50%> 

In [17]:
#실습
from fastapi import FastAPI
import nest_asyncio
import uvicorn

# 웹 서버 초기화
app = FastAPI()
nest_asyncio.apply()

# 라우터 설정(get방식 , root)
@app.get("/hello")
async def root():
    return{"message":"야이야"}

# 메인함수라면 (파이썬을 단독으로 실행했다면)
if __name__ == "__main__":
    # 서버시작
    uvicorn.run(app, host="218.149.140.96",port=9000)

INFO:     Started server process [443456]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://218.149.140.96:9000 (Press CTRL+C to quit)


INFO:     218.149.140.96:55703 - "GET / HTTP/1.1" 404 Not Found
INFO:     218.149.140.96:55704 - "GET /hello HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [443456]


### URL을 변수로 사용하기

- <font color=red>@app.get("/hello/{value}")</font> 
  - URL에  value 값을 설정
  - 브라우저에서 IP와 포트로 서버 접속하고 URL로 100을 넘김
    
  <img src="./lecture_image/04_fastapi03.png" width=50%>     

In [19]:
#실습
from fastapi import FastAPI
import nest_asyncio
import uvicorn

# 웹 서버 초기화
app = FastAPI()
nest_asyncio.apply()

# 라우터 설정(get방식 , root)
# value : 클라이언트에서 넘긴 값을 받 는 변수
@app.get("/hello/{value}")
async def root(value):
    return{"message" : value}

# 메인함수라면 (파이썬을 단독으로 실행했다면)
if __name__ == "__main__":
    # 서버시작
    uvicorn.run(app, host="218.149.140.96",port=9000)

INFO:     Started server process [443456]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://218.149.140.96:9000 (Press CTRL+C to quit)


INFO:     218.149.140.96:55759 - "GET / HTTP/1.1" 404 Not Found
INFO:     218.149.140.96:55760 - "GET /hello/%EA%B9%80%EB%8F%85%EC%9E%90 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [443456]


### html 문서를 반환하기

#### 코드 형태로 반환하기

- <font color=red>@app.get("/", response_class=HTMLResponse)</font>
  - HTML 문서를 반환

- 브라우저에서 IP와 포트로 서버 접속

  <img src="./lecture_image/04_fastapi04.png" width=50%>  

In [26]:
#실습
from fastapi import FastAPI
import nest_asyncio
import uvicorn
from fastapi.responses import HTMLResponse

# 웹 서버 초기화
app = FastAPI()
nest_asyncio.apply()

# 라우터 설정(get방식 , root)
# value : 클라이언트에서 넘긴 값을 받 는 변수
@app.get("/", response_class=HTMLResponse)
async def root():
    return """
    <html><body>
        <h1>안녕하세요 !! </h1>
    </body></html>
    """

# 메인함수라면 (파이썬을 단독으로 실행했다면)
if __name__ == "__main__":
    # 서버시작
    uvicorn.run(app, host="218.149.140.96",port=9000)

INFO:     Started server process [443456]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://218.149.140.96:9000 (Press CTRL+C to quit)


INFO:     218.149.140.96:55907 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [443456]


#### 파일 형태로 반환하기

- <font color=red>@app.get("/", response_class=FileResponse)</font>
  - 파일 문서를 반환
  - templates 폴더에 저장
  
  -%%writefile 명령 위쪽에는 아무 내용도 있으면 안됨(무조건 첫 줄에 사용)
      - 해당 명령 다음에 있는 내용을 파일에 저장하는 기능

In [28]:
%%writefile ./templates/index.html
<html><body>
    <h2>안녕하세요 !! <br>FAST API입니다 </h2>
</body></html>

Overwriting ./templates/index.html


In [29]:
#실습
from fastapi import FastAPI
import nest_asyncio
import uvicorn
from fastapi.responses import FileResponse

# 웹 서버 초기화
app = FastAPI()
nest_asyncio.apply()

# 라우터 설정(get방식 , root)
# value : 클라이언트에서 넘긴 값을 받 는 변수
@app.get("/", response_class=FileResponse)
async def root():
    return "templates/index.html"

# 메인함수라면 (파이썬을 단독으로 실행했다면)
if __name__ == "__main__":
    # 서버시작
    uvicorn.run(app, host="218.149.140.96",port=9000)

INFO:     Started server process [443456]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://218.149.140.96:9000 (Press CTRL+C to quit)


INFO:     218.149.140.96:56031 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [443456]


### 이미지가 포함된 HTML 문서 반환하기

- static 폴더 : 자원을 담아 놓는 폴더
- static 폴더에 이미지 폴더를 만들고 출력할 이미지를 저장

- jinja2 : HTML 문서에서 파이썬 코드를 사용하도록 지원하는 라이브러리 (JSP와 유사)

In [30]:
!pip install jinja2

In [41]:
%%writefile ./templates/index2.html
<html><body>
    <h2>안녕하세요 !! <br>FAST API입니다 </h2>
    <img src="{{url_for('static', path='save_1.png')}}"
</body></html>

Overwriting ./templates/index2.html


- <font color=red>Jinja2Templates(directory="templates/")</font>

  - Jinja2 템플릿 사용
  
  
- <font color=red>app.mount("/static", StaticFiles(directory="static", html=True), name="static")</font>  

  - "/static" : 마운트할 경로 
  - directory="static" : 정적파일이 들어 있는 폴더명
  - html=True : HTML 문서 여부
  - name="static" : FastAPI에서 내부적으로 사용할 이름
  
  
- <font color=red>templates.TemplateResponse("imgdisp2.html", {"request": request})  </font>  

  - Jinja 템플릿으로 구성된 HTML 문서를 request한 위치로 반환

In [42]:
#실습
from fastapi import FastAPI, Request
import nest_asyncio
import uvicorn
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles

# 웹 서버 초기화
app = FastAPI()
nest_asyncio.apply()

# Jinja 템플릿이 포함된 HTML 문서의 위치
templates = Jinja2Templates(directory="templates/")
# 이미지의 위치
app.mount("/static",StaticFiles(directory="static",html=True),name="static")


# 라우터 설정(get방식 , root)
# value : 클라이언트에서 넘긴 값을 받 는 변수
@app.get("/", response_class=HTMLResponse)
async def root(request : Request):
    # 클라이언트(request)를 설정
    return templates.TemplateResponse("index2.html",{"request":request})

# 메인함수라면 (파이썬을 단독으로 실행했다면)
if __name__ == "__main__":
    # 서버시작
    uvicorn.run(app, host="218.149.140.96",port=9000)

INFO:     Started server process [443456]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://218.149.140.96:9000 (Press CTRL+C to quit)


INFO:     218.149.140.96:56269 - "GET / HTTP/1.1" 200 OK
INFO:     218.149.140.96:56269 - "GET /save_1.png HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [443456]


# 데이터 스트리밍 구현

## 구현 방법

- <font color=red>StreamingResponse(test())</font>
  - 스트리밍 함수로 test()를 실행
  
  <img src="./lecture_image/04_fastapi05.png" width=50%>  

In [13]:
#실습

- yield() 함수 배우기

  
  <img src="./lecture_image/04_yield.png" width=70%>  

In [14]:
#실습

In [15]:
#실습 

## 웹 브라우저에 카메라 영상 출력하기

- index.html를 작성하고 templates 폴더에 저장
  - video_feed : 실행할 함수명
  
  
- Jinja 템플릿
    - 간단한 문법으로 html에서 {}, {{}} 등의 규칙을 이용하여 python 프로그래밍이 가능
    - 데이터를 담은 변수를 return값으로 html과 함께 넘겨준 뒤, html에서 jinja 템플릿 규칙에 따라 화면에 표현할 수 있음
    - Jinja 템플릿 html에서 주석은 {# 주석으로 처리할 내용 #}으로 표시
    - Jinja2 Tempaltes 변수 넘기기 : {{ 변수명 }}
    
  - 참고 
    - https://velog.io/@jewon119/01.Flask-기초-Jinja-template 
    - https://jinja.palletsprojects.com/en/2.11.x/templates/
  
  
- <font color=red>url_for()</font> : URL 주소값을 가져온다

In [44]:
%%writefile ./templates/index3.html
<html><body>
    <h2>CCTV 영상</h2>
    <img src="{{url_for('video_feed')}}"
</body></html>

Overwriting ./templates/index3.html


- <font color=red>ret, buffer = cv2.imencode('.jpg', frame)</font> : frame 이미지를 jpg로 인코딩

- <font color=red>frame = buffer.tobytes()</font> : 전송을 위해 인코딩된 이미지를 byte 형식으로 변환


- <font color=red>yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n') </font>

  - b : byte 형식임을 의미
  - --frame : 프레임을 표시
  - Content-Type: image/jpeg : 문서가 jpg 이미지임을 표시
  
  
- <font color=red>StreamingResponse(get_frames(), media_type="multipart/x-mixed-replace; boundary=frame")</font>

  - get_frames() : 호출할 함수명
  - mimetype : 클라이언트에게 전송된 문서의 타입을 알려주기 위한 파라미터 (type/subtype)
  - multipart : 복합문서 타입 (파일, 영상 등)을 의미
  - x-mixed-replace : x (추가적인 확장 형식), mixed (복합문서), repalce (subtype을 다음 메시지로 대체)
  - boundary : 복합문서 내의 각 문서들을 구분하는 분리자 (동영상이므로 frame으로 구분)

- 이미지 전송 부분을 동기방식으로 구현하는 경우

In [17]:
#실습

- 이미지 전송 부분을 비동기 방식으로 구현한 경우

In [7]:
#실습
import cv2
import nest_asyncio
import asyncio
import uvicorn
from fastapi import FastAPI, Request
from fastapi.templating import Jinja2Templates
from fastapi.responses import StreamingResponse

app = FastAPI()
nest_asyncio.apply()
templaes =Jinja2Templates(directory='templates/')

cap = cv2.VideoCapture("images/video.mp4")

async def get_frames():
    while cap.isOpened():
        ret, frame = cap.read()
        cv2.waitKey(33)
        
        if not ret:
            continue
        else:
            _, buffer = cv2.imencode(".jpg", frame)
            frame = buffer.tobytes()
        
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.get("/")
async def root(request : Request):
    return templaes.TemplateResponse("index3.html", {"request" : request})

@app.get("/video_feed")
async def video_feed():
    return StreamingResponse(get_frames(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    uvicorn.run(app, host="218.149.140.96",port=9000)

C:\Users\smhrd\anaconda3\lib\logging\__init__.py:632: RuntimeWarning: coroutine 'Server.serve' was never awaited
  return self._style.usesTime()
INFO:     Started server process [9728]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://218.149.140.96:9000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9728]


## 웹 출력 영상을 Binary 이미지로 출력하기

In [4]:
%%writefile ./templates/index4.html
<html><body>
<h3>웹 카메라 구현(원영상, 이진영상)</h3>
<img src="{{url_for('video_feed1')}}">
</body></html>

Overwriting ./templates/index4.html


In [1]:
#실습
import cv2

cap1 = cv2.VideoCapture("./images/video.mp4")

async def get_frames1() :
    while cap1.isOpened() :
        ret, frame = cap1.read()
        cv2.waitKey(33)
        
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.adaptiveThreshold(frame_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                     cv2.THRESH_BINARY, 9, 5)
        if not ret :
            continue
        else :
            _, buffer = cv2.imencode(".jpg", frame)
            frame = buffer.tobytes()
       
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')        

    cap1.release()   

In [5]:
import nest_asyncio
import asyncio
import uvicorn
from fastapi import FastAPI, Request
from fastapi.templating import Jinja2Templates
from fastapi.responses import StreamingResponse

app = FastAPI()
nest_asyncio.apply()
templates = Jinja2Templates(directory="templates/")

@app.get("/")        
async def root(request : Request) :
    return templates.TemplateResponse("index4.html", {"request" : request})

@app.get("/video_feed1") 
async def video_feed1() :
    return StreamingResponse(get_frames1(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__" :
    uvicorn.run(app, host="218.149.140.96", port=9000)

INFO:     Started server process [8848]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://218.149.140.96:9000 (Press CTRL+C to quit)


INFO:     218.149.140.96:51032 - "GET / HTTP/1.1" 200 OK
INFO:     218.149.140.96:51032 - "GET /video_feed1 HTTP/1.1" 200 OK


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 407, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\smhrd\anaconda3\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\smhrd\anaconda3\lib\site-packages\fastapi\applications.py", line 271, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\smhrd\anaconda3\lib\site-packages\starlette\applications.py", line 118, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\smhrd\anaconda3\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "C:\Users\smhrd\anaconda3\lib\site-packages\starlette\middleware\errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "C:\Users\smhrd\ana

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; text-align: left;"><font size=4 color=red><b>실습문제</b></font><br><br>
        <font size=4>
○ 이진 이미지를 출력하는 웹 카메라 만들기<br>
   - threshold() 함수 적용<br>
   - OTSU 이진화 알고리즘 적용<br>
   - 적응형 thresholding 적용</font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;">
        <img src="./lecture_image/04_flask06.png" width=30%><img src="./lecture_image/04_flask07.png" width=30%></td></tr>   
</table>

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:40px; text-align: center;"><font size=4 color=blue><b>[4차시] 학습요약</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>
        
○ <font color=red>ret, buffer = cv2.imencode()</font> : 영상 인코딩 함수<br>
○ yield() : 반복 실행 중에 중간 과정을 반환할 때 사용<br>

○ FastAPI : 파이썬으로 제작된 웹 프레임워크 (DJango의 축소 버전)<br>
        
○ <font color=red>app = FastAPI()</font> : FastAPI 객체를 app 변수에 할당<br>
○ <font color=red>nest_asyncio.apply()</font> : FastAPI와 uvicorn 연결<br>
○ <font color=red>@app.get("/")</font> : FastAPI에게 어떤 URL이 get방식으로 해당 함수를 실행하는지 알려줌<br>
○ <font color=red>@app.get("/hello")</font> : 서버 주소에 추가적인 URL을 설정<br>
○ <font color=red>@app.get("/hello/{value}")</font> : URL에 value 값을 설정<br>
○ <font color=red>uvicorn.run(app, host="127.0.0.1", port=9600)</font> : 서버 IP, 포트 등을 설정하고 서버를 실행<br>

○ <font color=red>HTMLResponse()</font> : html 문서를 반환하기<br>
○ <font color=red>FileResponse()</font> : 파일로 된 문서를 반환하기<br>
○ <font color=red>TemplateResponse()</font> : 템플릿 형태로 된 문서를 반환하기<br>
○ <font color=red>StreamingResponse(stream_with_context())</font> : 데이터 스트리밍을 반환하기
        
</font></td></tr>   
</table>